In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt
boston = load_boston()
print(boston.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])


In [2]:
train_data,test_data,train_target,test_target = train_test_split(boston.data,boston.target,test_size=0.2,random_state=42)
print(train_data.shape, train_target.shape, test_data.shape, test_target.shape)
print(train_data.__class__)

(404, 13) (404,) (102, 13) (102,)
<class 'numpy.ndarray'>


In [3]:
lr=make_pipeline(StandardScaler(), LinearRegression())
ridge=make_pipeline(StandardScaler(), Ridge(alpha=0.01))
lasso=make_pipeline(StandardScaler(), Lasso(alpha=0.01))
tree=make_pipeline(StandardScaler(), DecisionTreeRegressor(random_state=0))
svm = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
nn = make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(100,),random_state=1,max_iter=1000))

lr.fit(train_data,train_target)
ridge.fit(train_data,train_target)
lasso.fit(train_data,train_target)
tree.fit(train_data,train_target)
svm.fit(train_data,train_target)
nn.fit(train_data,train_target)

print("train set")
print(lr.score(train_data,train_target))
print(ridge.score(train_data,train_target))
print(lasso.score(train_data,train_target))
print(tree.score(train_data,train_target))
print(svm.score(train_data,train_target))
print(nn.score(train_data,train_target))
print("test set")
print(lr.score(test_data,test_target))
print(ridge.score(test_data,test_target))
print(lasso.score(test_data,test_target))
print(tree.score(test_data,test_target))
print(svm.score(test_data,test_target))
print(nn.score(test_data,test_target))

train set
0.7508856358979673
0.7508856342719583
0.7508035627152096
1.0
0.6843455679106752
0.8747082871418946
test set
0.6687594935356318
0.6687565288011177
0.6681815922762605
0.862408597996961
0.6509027052149564
0.8261315025115505
